In [2]:
import os 
import format_hansard
import pandas as pd
import json
import gensim 
import sklearn
from gensim.models import Word2Vec
from tqdm import tqdm
import string
import copy
import nltk
tqdm.pandas()
import spacy
nlp = spacy.load('en_core_web_sm')

In [50]:
from platform import python_version 
print(python_version())

3.8.12


# Scanning Files
1) Run scan_folder on the exact directory you've saved the Hansard files

In [ ]:
nltk.download('stopwords')

In [46]:
files = []
def scan_folder(parent):
    """
    Collects relevant file names from data folder

    Inputs:
        parent - parent file directory to start your search;
        shouuld end with .../hansard_full 2/

    Returns:
        List of filenames
    """
    # iterate over all the files in directory 'parent'
    for file_name in os.listdir(parent):
        if file_name.endswith(".txt"):
            # if it's a txt file, print its name (or do whatever you want)
            files.append("".join((parent, "/", file_name)))
        else:
            current_path = "".join((parent, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recursively call this method
                scan_folder(current_path)
    return files

In [4]:
def word_tokenize(word_list):
    """
    Turns string of words into a list of tokens
    
    Input: 
        word_list - list of strings 
    
    Returns: 
        list of tokens
    """
    tokenized = []
    # pass word list through language model.
    doc = nlp(word_list)
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            tokenized.append(token.text)
    return tokenized

In [ ]:
def normalizeTokens(word_list, extra_stop=[]):
    """
    Normalizes the tokenized sentences

    Inputs:
        word_list: list of tokenized words

    Returns:
        List of normalized words
    """
    # We can use a generator here as we just need to iterate over it
    normalized = []
    if type(word_list) == list and len(word_list) == 1:
        word_list = word_list[0]

    if type(word_list) == list:
        word_list = ' '.join([str(elem) for elem in word_list])

    doc = nlp(word_list.lower())

    # add the property of stop word to words considered as stop words
    if len(extra_stop) > 0:
        for stopword in extra_stop:
            lexeme = nlp.vocab[stopword]
            lexeme.is_stop = True

    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and len(
                w.text.strip()) > 0:
            # we add the lematized version of the word
            normalized.append(str(w.lemma_))

    return [[x] for x in normalized]


In [5]:
def migrant_worker(y): 
    """
    Converts 'migrant worker' to one token 
    
    Input: 
        y - list of strings
    
    Returns: 
        string with migrant worker replaced
    """
    if type(y) == float: 
        y = str(y)
    if 'migrant worker' in y:
        return '. '.join([x.replace('migrant worker', 'migrant-worker') for x in y.split('. ') if 'migrant worker' in x])
    else: 
        return y

In [6]:
def clean(df):
#     punc = '!()-[]{};:'"\,<>./?@#$%^&*_~'
    df = pd.concat(df, ignore_index=True)
    df = df.dropna()
    df['clean'] = df[df.iloc[:,0].str.len() >= 20]
    df['clean'] = df['clean'].apply(lambda x: migrant_worker(x))
    df['clean'] = df['clean'].progress_apply(lambda x: ' '.join(x.lower().translate(str.maketrans('', '', string.punctuation))
                                    .split() if type(x)==str else x))
    df['tokenized'] = df['clean'].progress_apply(lambda x: word_tokenize(x))
    
    return df.dropna().reset_index()
#     df.to_csv('/Users/Administrator/Desktop/UChicago/Y1Q3/30200/Project/hansard_data/hansard_full 2/{}'.format(df[3:]))

In [7]:
# files = scan_folder('/Users/Administrator/Desktop/UChicago/Y1Q3/30200/Project/hansard_data/hansard_full 2')

In [28]:
def create_df(files):
    pd12 = []
    pd13 = []
    pd14 = []
    pd15 = []
    pd16 = []
    pd17 = []
    pd18 = []
    pd19 = []
    pd20 = []
    pd21 = []
    massive = []
    for file in tqdm(files):
        if '2012' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd12.append(pd.DataFrame(lis))
        elif '2013' in file: 
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd13.append(pd.DataFrame(lis))
        elif '2014' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd14.append(pd.DataFrame(lis))
        elif '2015' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd15.append(pd.DataFrame(lis))
        elif '2016' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd16.append(pd.DataFrame(lis))
        elif '2017' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd17.append(pd.DataFrame(lis))
        elif '2018' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd18.append(pd.DataFrame(lis))
        elif '2019' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd19.append(pd.DataFrame(lis))
        elif '2020' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd20.append(pd.DataFrame(lis))
        elif '2021' in file:
            f = open(file, 'r')
            lis = [x.lower().replace('\xa0','').replace('\n','').split(':  ') for x in f.readlines()]
            f.close()
            pd21.append(pd.DataFrame(lis))
        
    massive.append(clean(pd12))
    massive.append(clean(pd13))
    massive.append(clean(pd14))
    massive.append(clean(pd15))
    massive.append(clean(pd16))
    massive.append(clean(pd17))
    massive.append(clean(pd18))
    massive.append(clean(pd19))
    massive.append(clean(pd20))
    massive.append(clean(pd21))
    
    for i, file in zip(range(12,22), massive): 
        file.to_csv('/Users/Administrator/Desktop/UChicago/Y1Q3/30200/Project/hansard_data/hansard_full 2/new_data1/{}.csv'.format(i))

In [47]:
# files = scan_folder('/Users/Administrator/Desktop/UChicago/Y1Q3/30200/Project/hansard_data/hansard_full 2')
create_df(scan_folder('/Users/Administrator/Desktop/UChicago/Y1Q3/30200/Project/hansard_data/hansard_full 2'))


  5%|█▊                                     | 284/5947 [00:02<00:55, 101.91it/s]


KeyboardInterrupt: 

In [ ]:
def create_final_df(path, savepath):
    speeches = 0
    final_df = pd.DataFrame(columns = ['year', '0', 'clean', 'tokenized'])
    for i in range(12,22):
        file = pd.read_csv(path + '{}.csv'.
                             format(i))
        speeches += len(file)
        file['year'] = int('20{}'.format(i))
        file = file[['year', '0', 'clean', 'tokenized']]
        final_df = pd.concat([final_df, file], ignore_index = True)
    
    final_df = final_df.dropna()
    final_df.to_csv(savepath)
